In [1]:
from timeit import default_timer as timer
import socket
import sys
import os
import numpy as np
import pandas as pd
import re
import string
import reverse_geocoder as rg
from iso3166 import countries
from glob import glob

In [2]:
# Local
print('Hostname:', socket.gethostname())
if 'samuel' in socket.gethostname().lower():
    path_to_data='../../data/tweets/tweets-with-geocoordinates-or-place/extract'
# Cluster
else:
    path_to_data='/scratch/spf248/twitter/data/tweets/tweets-with-geocoordinates-or-place/extract'

print(path_to_data)

Hostname: Samuels-MacBook-Pro.local
../../data/decahose/parsed/tweets/tweets-with-geocoordinates-or-place-extract


In [3]:
print("LOAD")
tweets=spark.read.parquet(path_to_data)

LOAD
# Tweets: 7560


In [4]:
print("CACHE")
tweets.cache()

DataFrame[tweet_id: string, created_at: timestamp, text: string, tweet_lang: string, user_id: string, user_location: string, place_id: string, tweet_longitude: double, tweet_latitude: double]

In [8]:
print("Drop Missing Geocoordinates...")
tweets=tweets.filter((tweets["tweet_latitude"].isNotNull())&(tweets["tweet_longitude"].isNotNull()))

Drop Missing Geocoordinates...


1131

In [9]:
rg.search((0,0))

Loading formatted geocoded file...


[OrderedDict([('lat', '4.88447'),
              ('lon', '-1.75536'),
              ('name', 'Takoradi'),
              ('admin1', 'Western'),
              ('admin2', ''),
              ('cc', 'GH')])]

In [ ]:
def reverse_geocode(x,y):
    return abs(x-y)

udf_dict = udf(dict, IntegerType())

In [11]:
print("Reverse geocode representative point...")
start = timer()

tweets['tweet_point'] = list(zip(tweets['tweet_latitude'], tweets['tweet_longitude']))

reversed_geocodes = pd.DataFrame(rg.search(list(tweets['tweet_point'].values))).drop(['lat','lon'],1)

reversed_geocodes['country'] = reversed_geocodes['cc'].apply(lambda x:countries.get(x).name)

tweets = pd.concat([tweets.drop('tweet_point',1),reversed_geocodes.rename(columns=lambda x:'tweet_'+x)],1)

del reversed_geocodes

print("Done in", round(timer()-start), "sec")

Reverse geocode representative point...
Done in 0 sec


In [12]:
tweets.head()

,tweet_id,created_at,text,tweet_lang,user_id,user_location,place_id,tweet_longitude,tweet_latitude,tweet_name,tweet_admin1,tweet_admin2,tweet_cc,tweet_country
0,812406347430031361,2016-12-23 21:15:41,Almuerzo navideño (@ Mar y Punto in Miraflores...,es,79900019,"Lima, Peru",005bf2190435cf68,-77.009212,-12.130520,Surco,Lima,Lima,PE,Peru
1,812411665807507456,2016-12-23 21:36:49,#Veiomever\n@dihanfreitas @ Academia Norpra Fí...,pt,389819773,None,37b82f7e75d4f3b0,-43.034610,-22.650760,Guapimirim,Rio de Janeiro,Guapimirim,BR,Brazil
2,812425498617753601,2016-12-23 22:31:47,Want to work at Mercy? We're #hiring in #Festu...,en,1898258479,None,cace34546a1607a9,-90.395954,38.220610,Festus,Missouri,Jefferson County,US,United States of America
3,812425884476981249,2016-12-23 22:33:19,Viernes de tranqui 🍹🍻☕ @ El Cuélebre Negro htt...,es,3169714036,"Langreo, España",fac43a227940d7b2,-5.689620,43.311560,Sama,Asturias,Province of Asturias,ES,Spain
4,812431567750660097,2016-12-23 22:55:54,R-R-Reunited pt.2\n#iceskating #missedyou #did...,es,439148331,Barcelona,1a27537478dd8e38,2.119873,41.380644,les Corts,Catalonia,Provincia de Barcelona,ES,Spain
